In [ ]:
import os
import sys
 
spark_home = '/opt/cloudera/parcels/SPARK2/lib/spark2'
os.environ['SPARK_HOME'] = spark_home
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python'
os.environ['LD_LIBRARY_PATH'] = '/opt/python/virtualenv/jupyter/lib'
os.environ['PYSPARK_PYTHON'] = '/opt/cloudera/parcels/PYENV.ZNO0059623792/bin/python'
 
sys.path.insert(0, os.path.join (spark_home,'python'))
 
#sys.path.insert(0, os.path.join (spark_home,'python/lib/py4j-0.10.7-src.zip'))
sys.path.insert(0, os.path.join (spark_home,'python/lib/py4j-0.10.4-src.zip'))
from pyspark import SparkContext, SparkConf, HiveContext
 
conf = SparkConf().setAppName('SparkContextExample')\
    .setMaster("yarn-client")\
    .set('spark.dynamicAllocation.enabled', 'false')\
# Пример кастомизации параметров Spark. По умолчанию параметры берутся из конфигурации Cloudera Manager.
# .set('spark.local.dir', 'sparktmp')\ # Директория для временных файлов Spark
# .set('spark.executor.memory','5g')\ # Объём памяти для выполнения задач Spark
# .set('spark.driver.maxResultSize','5g')\ # Максимальный объём памяти для драйвера Spark
# .set('spark.executor.instances', '1')\ # Количество экземпляров/инстансов для выполнения задач Spark
# .set('spark.executor.cores', '8')\ # Количество ядер, задействованных для выполнения задач Spark
# .set('spark.port.maxRetries', '150')\ # Максимальное количество попыток привязки к порту до отказа


In [6]:
import pandas as pd
import re
import numpy as np
from datetime import datetime as dt

from h3 import h3

pd.set_option('display.float_format', '{:.4f}'.format)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [8]:
dir(h3)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_api_functions',
 '_cy',
 '_globals',
 '_in_collection',
 '_in_scalar',
 '_out_ordered',
 '_out_scalar',
 '_out_unordered',
 'cell_area',
 'compact',
 'edge_length',
 'exact_edge_length',
 'experimental_h3_to_local_ij',
 'experimental_local_ij_to_h3',
 'geo_to_h3',
 'get_destination_h3_index_from_unidirectional_edge',
 'get_h3_indexes_from_unidirectional_edge',
 'get_h3_unidirectional_edge',
 'get_h3_unidirectional_edge_boundary',
 'get_h3_unidirectional_edges_from_hexagon',
 'get_origin_h3_index_from_unidirectional_edge',
 'get_pentagon_indexes',
 'get_res0_indexes',
 'h3_distance',
 'h3_get_base_cell',
 'h3_get_faces',
 'h3_get_resolution',
 'h3_indexes_are_neighbors',
 'h3_is_pentagon',
 'h3_is_res_class_III',
 'h3_is_res_class_iii',
 'h3_is_valid',
 'h3_line',
 'h3_set_to_multi_polygon',
 'h3_to_center_child',
 'h3_to_children',
 'h3_to_geo',
 'h3_to_geo_boundary',
 '

In [17]:
#h3.cell_area(3)
h3.edge_length(3)

59.81085794

In [ ]:
h3.h3_to_parent('8910c264d73ffff', 6)

In [16]:
census_h3 = pd.read_excel('test.xlsx')
census_h3['h3_id_8'] = census_h3.apply(lambda x:
                                      h3.h3_to_parent(x['h3_id'], 
                                      8),
                                      axis=1)
census_h3['h3_id_7'] = census_h3.apply(lambda x:
                                      h3.h3_to_parent(x['h3_id'], 
                                      7),
                                      axis=1)
census_h3['h3_id_6'] = census_h3.apply(lambda x:
                                      h3.h3_to_parent(x['h3_id'], 
                                      6),
                                      axis=1)
census_h3['h3_id_5'] = census_h3.apply(lambda x:
                                      h3.h3_to_parent(x['h3_id'], 
                                      5),
                                      axis=1)
census_h3['h3_id_4'] = census_h3.apply(lambda x:
                                      h3.h3_to_parent(x['h3_id'], 
                                      4),
                                      axis=1)
census_h3['h3_id_3'] = census_h3.apply(lambda x:
                                      h3.h3_to_parent(x['h3_id'], 
                                      3),
                                      axis=1)
census_h3.to_excel('trans_hexes.xlsx', index=None)                                                       

In [ ]:
print('Start',dt.now())
sc = SparkContext.getOrCreate(conf=conf)
print('SparkContext start',dt.now())
hive = HiveContext(sc)
print('Allocated', dt.now())

In [ ]:
trans = hive.read.parquet('hdfs://clsklcib/data/custom/cib/mntz/pa/card_transactions/partition_dt=2020-11-01')

In [ ]:
#собираем справочник по типам оперций
trans.select('trans_type', 'trans_name').drop_duplicates(['trans_type', 'trans_name']).toPandas().to_excel('trans_spr.xlsx', index=None)

In [ ]:
#собираем список городв Екб
city_list = trans.select('trans_city').filter(trans['trans_city'].contains("EKATERINBUR")).drop_duplicates(['trans_city'])

In [ ]:
#собираем все POS-транзакции в Екб
ekb_trans = trans.filter('trans_type = 1010 or trans_type = 1310').join(city_list, on=['trans_city'], how='inner')

#собираем id пользователей, топ1000 по кол-во транзакций
users = ekb_trans.select('target_client_id').groupby('target_client_id').count().sort(F.desc('count')).limit(1000)

In [ ]:
#Выбираем из всех транзакций те, которые были соверешены нашим топ1000
df = trans.filter('trans_type = 1010 or trans_type = 1310').join(users, on=['target_client_id'], how='inner')
df.sort('target_client_id').toPandas()

In [ ]:
df.write.parquet('hdfs://clsklsbx/user/team/team_digitcamp/hive/tmp_mon_trans_top1000')

In [ ]:
import datetime as dt

for d in range(2,31):
    day = str(dt.date(2020,11,d))
    l_trans = hive.read.parquet('hdfs://clsklcib/data/custom/cib/mntz/pa/card_transactions/partition_dt=' + day)
    l_df = l_trans.filter('trans_type = 1010 or trans_type = 1310').join(users, on=['target_client_id'], how='inner')
    l_df.write.mode('append').parquet('hdfs://clsklsbx/user/team/team_digitcamp/hive/tmp_mon_trans_top1000')
    print('Done: ', day)

In [ ]:

#Работа с собранной ранее базой топ1000

In [ ]:
top1000 = hive.read.parquet('hdfs://clsklsbx/user/team/team_digitcamp/hive/tmp_mon_trans_top1000')

In [ ]:
sensus = hive.read.parquet('hdfs://clsklcib/data/custom/cib/mntz/pa/census_ext_w4/partition_dt=2020-11-01')
cols = ['terminal_id', 'latitude_mt', 'longitude_mt']
sensus_terminals = sensus.select(cols).filter(sensus['province_mt'].contains("Свердловская")).filter('latitude_mt is not null and longitude_mt is not null ').drop_duplicates(['terminal_id'])

#преобразуем char-координаты в decimal 
#sensus_terminals = sensus_terminals.withColumn('latitude_mt_n', F.col('latitude_mt').cast(FloatType())).withColumn('longitude_mt_n', F.col('longitude_mt').cast(FloatType()))

In [ ]:
sensus_terminals_pd = sensus_terminals.toPandas()
#добавляем координаты в числовом виде
sensus_terminals_pd['lat_n'] = pd.to_numeric(sensus_terminals_pd['latitude_mt'], downcast='float')
sensus_terminals_pd['lng_n'] = pd.to_numeric(sensus_terminals_pd['longitude_mt'], downcast='float')


In [ ]:
sensus_terminals_pd['h3_id'] = sensus_terminals_pd.apply(lambda x:
                                                        h3.geo_to_h3(x['lat_n'], 
                                                                     x['lng_n'], 9),
                                                        axis=1)

sensus_terminals_pd[['terminal_id', 'lat_n', 'lng_n', 'h3_id']].to_excel('terminals_h3.xlsx', index=None)


In [ ]:
#траблы с версиями питона
#sensus_terminals_ps = hive.createDataFrame(sensus_terminals_pd[['terminal_id', 'lat_n', 'lng_n', 'h3_id']])
#sensus_terminals_ps.write.mode('append').parquet('hdfs://clsklsbx/user/team/team_digitcamp/hive/tmp_mon_trans_terminals_h3')

In [ ]:
sensus_terminals_pd

In [ ]:
#Считаем кол-во платежей за месяц в разрезе клиент-теримнал
cols = ['target_client_id','trans_dt','trans_time','trans_city','trans_amount','mcc_supercat','mcc_cat','mcc_name','merchant_id','merchant_name','terminal_id','partition_hr']
group_cols = ['target_client_id','mcc_supercat','mcc_cat','mcc_name','merchant_id','merchant_name','terminal_id']
top1000_grp = top1000.select(cols).groupby(group_cols).count()
top1000_grp.limit(8).toPandas()


In [ ]:
census = hive.createDataFrame(pd.read_excel('terminals_h3.xlsx'))

In [ ]:
census.limit(10).toPandas()

In [ ]:
census.write.parquet('hdfs://clsklsbx/user/team/team_digitcamp/hive/tmp_mon_trans_census_h3')

In [ ]:
#попытка преобразовать координаты во float (не работает из-за разных версий питона??)

def h3_9(lot, lng):
    return h3.geo_to_h3(lot, lng, 9)

udf_geo_to_h3 = F.udf(h3_9, StringType())
sensus_terminals.withColumn('h3_id', udf_geo_to_h3(sensus_terminals.latitude_mt_n,
                                                   sensus_terminals.longitude_mt_n)).limit(10).toPandas()

In [ ]:
client_h3 = top1000_grp.join(census, ['terminal_id']).select('target_client_id',
                                                                    'terminal_id',
                                                                    'mcc_name',
                                                                    'merchant_name',
                                                                    'h3_id',
                                                                    'count').sort('target_client_id')

In [ ]:
client_h3.write.parquet('hdfs://clsklsbx/user/team/team_digitcamp/hive/tmp_mon_trans_client_h3')

In [ ]:
client_h3 = hive.read.parquet('hdfs://clsklsbx/user/team/team_digitcamp/hive/tmp_mon_trans_client_h3')
client_h3.groupby('target_client_id','h3_id').agg(F.sum('count').alias('count')).sort('target_client_id', F.desc('count')).limit(100).toPandas()

In [ ]:
client_h3.limit(100).toPandas()

In [16]:
#функция для рисования гексов (не работает из альфы)
from h3 import h3
import folium

def visualize_hexagons(hexagons, color='red', folium_map=None):
    polylines = []
    lat = []
    lng= []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0] #список линий, которые формируют гекс
        lat.extend(map(lambda v:v[0], polyline)) #все широты в один список, для нахождения средней точки
        lng.extend(map(lambda v:v[1], polyline)) 
        polylines.append(polyline)
        
        if folium_map is None:
            m = folium.Map(locations=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=3, tiles='cartodbpositron')
        else:    
            m = folium_map 
        for polyline in polylines:
            my_Polyline = folium.PolyLine(locations=polyline, weight=3, color=color)
            m.add_child(my_Polyline)
    return m   
            
hex = h3.geo_to_h3(56.345833, 50.225562, 9)
m = visualize_hexagons([hex])
display(m)

In [10]:
dir (h3)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_api_functions',
 '_cy',
 '_globals',
 '_in_collection',
 '_in_scalar',
 '_out_ordered',
 '_out_scalar',
 '_out_unordered',
 'cell_area',
 'compact',
 'edge_length',
 'exact_edge_length',
 'experimental_h3_to_local_ij',
 'experimental_local_ij_to_h3',
 'geo_to_h3',
 'get_destination_h3_index_from_unidirectional_edge',
 'get_h3_indexes_from_unidirectional_edge',
 'get_h3_unidirectional_edge',
 'get_h3_unidirectional_edge_boundary',
 'get_h3_unidirectional_edges_from_hexagon',
 'get_origin_h3_index_from_unidirectional_edge',
 'get_pentagon_indexes',
 'get_res0_indexes',
 'h3_distance',
 'h3_get_base_cell',
 'h3_get_faces',
 'h3_get_resolution',
 'h3_indexes_are_neighbors',
 'h3_is_pentagon',
 'h3_is_res_class_III',
 'h3_is_res_class_iii',
 'h3_is_valid',
 'h3_line',
 'h3_set_to_multi_polygon',
 'h3_to_center_child',
 'h3_to_children',
 'h3_to_geo',
 'h3_to_geo_boundary',
 '

In [11]:
h3.edge_length(h3.h3_get_resolution(hex))

0.461354684

In [ ]:
sc.stop()